# BERT で始める分散機械学習

In [ ]:
# 環境ごとの値設定
group_name = "" # 共有でリソース使用時に重複を防止するための値、9文字まで
vnet_resourcegroup_name = ""
vnet_name = ""
subnet_name = ""

## Workspace の初期化

Azure Machine Learning Workspace リソースに接続し、Workspace オブジェクトを作成する。

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Compute Cluster の作成
Azure Machine Learning の管理下のクラスターである Computer Cluster を作成する。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = f"gpuclus{group_name}"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    # クラスター作成
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_NC6S_V3',
        max_nodes=2,
        idle_seconds_before_scaledown=300,
        vnet_resourcegroup_name=vnet_resourcegroup_name, # VNet 閉域化時のみ
        vnet_name=vnet_name, # VNet 閉域化時のみ
        subnet_name=subnet_name, # VNet 閉域化時のみ
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

## 学習
Compute Cluster にジョブを投入し、分散機械学習を実行する。

### Experiment の作成
実験の各試行をまとめて管理する [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) を作成する。

In [ ]:
from azureml.core import Experiment

experiment_name = f"pytorch-distributed-bert-fine-tuning-{group_name}"
experiment = Experiment(ws, name=experiment_name)

### Environment の作成

Environment の実態は Docker コンテナであり、下記サンプルの場合は environment.yml に記載されたパッケージがプリセットされたコンテナイメージが作成される。

初回実行時はイメージビルドが挟まるため非常に時間がかかるが、2回目以降はキャッシュされたコンテナイメージが使いまわされるためあまり時間がかからない。

実行時は Environment として保存されたコンテナイメージが Compute Cluster の各ノードに配られ、コンテナが立ち上がって Python コードを実行する。

In [ ]:
from azureml.core import Environment

environment_name = f"pytorch-distributed-env-{group_name}"
environment_file = "src/environment.yml"

pytorch_env = Environment.from_conda_specification(name=environment_name, file_path=environment_file)

## ベースとなる Docker イメージを指定
## 「Environment」にプリセットされた environment の詳細画面から確認可能
pytorch_env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu18.04:20211221.v1"
)
pytorch_env.register(workspace=ws)

### 学習ジョブの設定

Python スクリプトを実行する諸条件を定義する設定用オブジェクトである ScriptRunConfig に各種実行条件をセットする。分散機械学習の場合、```distributed_job_config```に分散機械学習用の設定ファイルを渡すことで、Compute Cluster が適切に初期化されて分散機械学習がスムーズに実行される。

今回は PyTorch DDP を使用しているため```PyTorchConfiguration```を渡しているが、OpenMPI 等ほかの分散機械学習の仕組みを使うことも可能。

```distributed_job_config```に渡す Configuration の種類に応じて、```NODE_RANK```のような依存する環境変数が自動でセットされたり、通信設定が裏で行われる。

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration

node_count = 2
arg = [
    "--batch_size", 1024,
    "--epochs", 40,
    "--lr", 1e-4,
    "--num_nodes", node_count,
    "--base_model", "cl-tohoku/bert-base-japanese-char-v2"
]

## cl-tohoku/bert-* から好きなものを選択
## https://huggingface.co/cl-tohoku

### ex) bert-large-japanese-char の場合
#arg = [
#    "--batch_size", 1024,
#    "--epochs", 40,
#    "--lr", 1e-5,
#    "--num_nodes", node_count,
#    "--base_model", "cl-tohoku/bert-large-japanese-char"
#]


dist_config = PyTorchConfiguration(node_count=node_count)

src = ScriptRunConfig(source_directory="src",
                      script='trainer_pytorch_distributed.py',
                      compute_target=compute_target,
                      environment=pytorch_env,
                      arguments=arg,
                      distributed_job_config=dist_config)

### ジョブの実行
Experiment に対してジョブを渡すことでジョブが実際に実行される。

Experiment 配下に Run が作成され、メトリックやログ等が自動で記録される。

In [ ]:
run = experiment.submit(src)
print(run)
run.wait_for_completion(show_output=True) # this provides a verbose log